# **Modelling and Tuning**

In [12]:
import pandas as pd
import numpy as np

In [13]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [14]:
import pickle

# Load train/val/test split data from notebook3
with open(r'Nata_Files\\train_test_split.pkl', 'rb') as f:
    notebook3_data = pickle.load(f)

# Core datasets
X = notebook3_data.get('X')
y = notebook3_data.get('y')

# Splits and processed feature sets
X_train = notebook3_data.get('X_train')
y_train = notebook3_data.get('y_train')
X_val = notebook3_data.get('X_val')
y_val = notebook3_data.get('y_val')
X_test = notebook3_data.get('X_test')
y_test = notebook3_data.get('y_test')
X_train_val = notebook3_data.get('X_train_val')
y_train_val = notebook3_data.get('y_train_val')

numeric_cols = notebook3_data.get('numeric_cols')
kf = notebook3_data.get('kf')
rkf = notebook3_data.get('rkf')
skf = notebook3_data.get('skf')

print("Train/Val/Test split data loaded successfully!")
if X is not None and y is not None:
    print(f"Full dataset X shape: {X.shape} | y shape: {y.shape}")
if X_train is not None:
    print(f"X_train shape: {X_train.shape}")
if X_val is not None:
    print(f"X_val shape: {X_val.shape}")
if X_test is not None:
    print(f"X_test shape: {X_test.shape}")
if kf is not None:
    try:
        print(f"kf splits: {kf.get_n_splits()}")
    except Exception:
        print("kf loaded (object), get_n_splits() unavailable for this object")
if rkf is not None:
    try:
        print(f"rkf splits: {rkf.get_n_splits()}")
    except Exception:
        print("rkf loaded (object), get_n_splits() unavailable for this object")

Train/Val/Test split data loaded successfully!
Full dataset X shape: (5196, 14) | y shape: (5196,)
X_train shape: (3117, 14)
X_val shape: (1039, 14)
X_test shape: (1040, 14)
kf splits: 10
rkf splits: 14


In [15]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import r2_score, accuracy_score

In [16]:
def fit_model(model, X, y):
    return model.fit(X, y)


In [8]:
def eval_clf_model(model, X, y):
    return model.score(X,y)

In [5]:
def avg_score_clf(method,X,y,model):
    score_train = []
    score_val = []
    for train_index, val_index in method.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        trained_model = fit_model(model, X_train, y_train)
        value_train = eval_clf_model(trained_model, X_train, y_train)
        value_val = eval_clf_model(trained_model, X_val, y_val)
        score_train.append(value_train)
        score_val.append(value_val)

    print('Train:', np.mean(score_train))
    print('Validation:', np.mean(score_val))

In [6]:
def avg_score_clf_skf(method, X, y, model):
    score_train = []
    score_val = []
    for train_index, val_index in method.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        trained_model = fit_model(model, X_train, y_train)
        value_train = eval_clf_model(trained_model, X_train, y_train)
        value_val = eval_clf_model(trained_model, X_val, y_val)
        score_train.append(value_train)
        score_val.append(value_val)

    print('Train:', np.mean(score_train))
    print('Validation:', np.mean(score_val))

## **Model Selection**

Que modelos podemos usar?
Isto é um classification problem então podemos descartar logo alguns.
Podemos testar :
- Logistic Regression
- Decision Trees
- Naive Bayes
- Random Forest
- K nearest Classifier

## **Model Evaluation**

#### **Logistic Regression**

In [17]:
logr = LogisticRegression()

In [10]:
logr_fit = logr.fit(X_train, y_train)

##### Hold-Out Method

In [11]:
train_acc_logr = logr_fit.score(X_train, y_train)
val_acc_logr = logr_fit.score(X_val, y_val)

print("Train:", str(train_acc_logr))
print("Validation:", str(val_acc_logr))

Train: 0.741738851459737
Validation: 0.7507218479307026


##### K-Fold CV

In [12]:
avg_score_clf(kf, X, y,logr)

Train: 0.7413181197396282
Validation: 0.7403768341485104


##### Repeated K-Fold CV

In [19]:
avg_score_clf(rkf, X, y,logr)

Train: 0.7422216979251883
Validation: 0.7410538404650568


##### Stratified K-Fold CV

In [20]:
avg_score_clf_skf(skf, X, y,logr)

Train: 0.7400993989296217
Validation: 0.7405717355861865


#### **Decision Tree Classifier**

In [15]:
dtc = DecisionTreeClassifier()
dtc_fit = dtc.fit(X_train, y_train)

##### Hold-Out Method

In [17]:
train_acc_dtc = dtc_fit.score(X_train, y_train)
val_acc_dtc = dtc_fit.score(X_val, y_val)

print("Train:", str(train_acc_dtc))
print("Validation:", str(val_acc_dtc))

Train: 1.0
Validation: 0.6948989412897016


##### K-Fold CV

In [18]:
avg_score_clf(kf, X, y, dtc)

Train: 1.0
Validation: 0.7003534904401956


##### Repeated K-Fold CV

In [21]:
avg_score_clf(rkf, X, y, dtc)

Train: 1.0
Validation: 0.6924524933262367


##### Stratified K-Fold CV

In [22]:
avg_score_clf_skf(skf, X, y, dtc)

Train: 1.0
Validation: 0.7009248554913295


#### **K-Neighbors Classifier**

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [29]:
knn_clf = KNeighborsClassifier()

knn_clf = knn_clf.fit(X_train, y_train)

##### Hold-Out Method

In [30]:
train_acc_knn = dtc_fit.score(X_train, y_train)
val_acc_knn = dtc_fit.score(X_val, y_val)

print("Train:", str(train_acc_knn))
print("Validation:", str(val_acc_knn))

Train: 0.3647738209817132
Validation: 0.3647738209817132


##### K-Fold CV

In [31]:
avg_score_clf(kf, X, y, knn_clf)

Train: 0.7965315954730326
Validation: 0.6941781532532978


##### Repeated K-Fold CV

In [32]:
avg_score_clf(rkf, X, y, knn_clf)

Train: 0.795194755260022
Validation: 0.6896602923655051


##### Stratified K-Fold CV

In [33]:
avg_score_clf_skf(skf, X, y, knn_clf)

Train: 0.796510223390843
Validation: 0.6968786127167629
